### **Fact Orders**

### Data Reading

In [0]:
df= spark.sql("select * from databrciks_catalog.silver.orders_silver")
df.display()

In [0]:
df_dimcustomer = spark.sql("select DimCustomerKey,customer_id as dim_customer_id from databrciks_catalog.gold.dimcustomers")
df_dimproducts = spark.sql("select product_id as DimProductKey,product_id as dim_product_id from databrciks_catalog.gold.dimproducts")

### Fact Dataframe

In [0]:
df_fact = df.join(df_dimcustomer,df['customer_id'] == df_dimcustomer['dim_customer_id'],how = 'left').join(df_dimproducts,df['product_id'] == df_dimproducts['dim_product_id'],how = 'left')

df_fact_new = df_fact.drop('dim_customer_id','dim_product_id','customer_id','product_id')

### Upsert On Fact

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databrciks_catalog.gold.FactOrders"):

    dlt_obj = DeltaTable.forName(spark, "databrciks_catalog.gold.FactOrders")

    dlt_obj.alias("trigger").merge(df_fact_new.alias("source"),"trigger.order_id = source.order_id AND trigger.DimCustomerKey = source.DimCustomerKey AND trigger.DimProductKey = source.DimProductKey"
    )\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()

else:
    df_fact_new.write.format("delta")\
        .option("path", "abfss://gold@avidatabricksete.dfs.core.windows.net/FactOrders")\
        .saveAsTable("databricks_cata.gold.FactOrders")
